# Gibbs sampling with PyMC3

## GitHub stars

- `PyMC3`
  - https://star-history.t9t.io/#pymc-devs/pymc3
- `TensorFlow probability`
  - https://star-history.t9t.io/#tensorflow/probability
- `Stan`
  - https://star-history.t9t.io/#stan-dev/stan
- `Edward`
  - https://star-history.t9t.io/#blei-lab/edward

(Checked on September 10, 2019)

## What they are for

As a tool for bayesian machine learning

## Getting started with PyMC3

https://docs.pymc.io/notebooks/getting_started

In [1]:
import numpy as np
import pymc3 as pm
import graphviz
from matplotlib import pyplot as plt

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
ERROR (theano.gpuarray): pygpu was configured but could not be imported or is too old (version 0.7 or higher required)
NoneType: None


ModuleNotFoundError: No module named 'graphviz'

In [ ]:
print('Running on PyMC3 v{}'.format(pm.__version__))

### Generating data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

# Initialize random number generator
np.random.seed(123)

# True parameter values
alpha, sigma = 1, 1
beta = [1, 2.5]

# Size of dataset
size = 100

# Predictor variable
X1 = np.random.randn(size)
X2 = np.random.randn(size) * 0.2

# Simulate outcome variable
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(size)*sigma

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True, figsize=(10,4))
axes[0].scatter(X1, Y)
axes[1].scatter(X2, Y)
axes[0].set_ylabel('Y'); axes[0].set_xlabel('X1'); axes[1].set_xlabel('X2');

### Model Specification

In [ ]:
# requirements: install `graphviz` in your system first. e.g. `sudo apt install graphviz`

graph = """
digraph {
  subgraph cluster_beta {
    beta;
  }
  
  subgraph cluster_x {
    x [style=filled, color=lightgrey];
  }
  
  alpha -> mu;
  beta -> mu;
  x -> mu;
  mu -> y;
  sigma -> y;
  y [style=filled, color=lightgrey];
}
"""
src = graphviz.Source(graph)
src

In [ ]:
basic_model = pm.Model()

with basic_model:

    # Priors for unknown model parameters
    alpha = pm.Normal('alpha', mu=0, sigma=10)
    beta = pm.Normal('beta', mu=0, sigma=10, shape=2)
    sigma = pm.HalfNormal('sigma', sigma=1)

    # Expected value of outcome
    mu = alpha + beta[0]*X1 + beta[1]*X2

    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal('Y_obs', mu=mu, sigma=sigma, observed=Y)

### Model fitting

In [ ]:
map_estimate = pm.find_MAP(model=basic_model, )

map_estimate

In [ ]:
map_estimate = pm.find_MAP(model=basic_model, method='powell')

map_estimate

In [ ]:
with basic_model:
    # draw 500 posterior samples
    trace = pm.sample(500)

NUTS(No U-Turn Sampler)

- Hamiltonian Monte Carlo Method + auto tuning of step size and number of steps
- does not use Metropolis acceptance
- only for continuous parameters which have gradients

References:

- https://besjournals.onlinelibrary.wiley.com/doi/full/10.1111/2041-210X.12681

In [ ]:
trace['alpha'][-5:]

In [ ]:
with basic_model:

    # instantiate sampler
    step = pm.Slice()

    # draw 5000 posterior samples
    trace = pm.sample(5000, step=step)

In [ ]:
pm.traceplot(trace);

In [ ]:
pm.summary(trace).round(2)

## What is Gibbs Sampling